In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose

In [40]:

df = pd.read_csv('test.csv')
dt = pd.read_csv('train.csv')
dfrs = pd.read_csv('features.csv')
ds = pd.read_csv('stores.csv')


In [41]:
print(df.head())
print(dt.head())
print(dfrs.head())
print(ds.head())

   Store  Dept        Date  IsHoliday
0      1     1  2012-11-02      False
1      1     1  2012-11-09      False
2      1     1  2012-11-16      False
3      1     1  2012-11-23       True
4      1     1  2012-11-30      False
   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False
   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        39.93       2.514        NaN        NaN   
3      1  2010-02-26        46.63       2.561        NaN        NaN   
4      1  2010-03-05        46.50       2.625        NaN        NaN   

   MarkDown3  MarkDown4  MarkDown

In [44]:
display(dt.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


None

In [46]:
dfrs.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [52]:
ds.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [50]:
dt.describe()

,Store,Dept,Weekly_Sales
count,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123
std,12.785297,30.492054,22711.183519
min,1.000000,1.000000,-4988.940000
25%,11.000000,18.000000,2079.650000
50%,22.000000,37.000000,7612.030000
75%,33.000000,74.000000,20205.852500
max,45.000000,99.000000,693099.360000


In [54]:
ds.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [56]:
dt.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

In [58]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


In [60]:
df.head()

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115064 entries, 0 to 115063
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Store      115064 non-null  int64 
 1   Dept       115064 non-null  int64 
 2   Date       115064 non-null  object
 3   IsHoliday  115064 non-null  bool  
dtypes: bool(1), int64(2), object(1)
memory usage: 2.7+ MB


In [64]:
display(ds['Type'].unique())

array(['A', 'B', 'C'], dtype=object)

In [66]:
display(dt['Dept'].unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 40, 41, 42, 44, 45, 46, 47, 48, 49, 51, 52, 54, 55, 56,
       58, 59, 60, 67, 71, 72, 74, 77, 78, 79, 80, 81, 82, 83, 85, 87, 90,
       91, 92, 93, 94, 95, 96, 97, 98, 99, 39, 50, 43, 65], dtype=int64)

In [68]:
dt['Store_Dept'] = dt['Store'].astype(str) + '_' + dt['Dept'].astype(str)
df['Store_Dept'] = df['Store'].astype(str) + '_' + df['Dept'].astype(str)

In [70]:
dt['Month'] = pd.to_datetime(dt['Date']).dt.month
dt['Year'] = pd.to_datetime(dt['Date']).dt.year

df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Year'] = pd.to_datetime(df['Date']).dt.year

dfrs['Month'] = pd.to_datetime(dfrs['Date']).dt.month
dfrs['Year'] = pd.to_datetime(dfrs['Date']).dt.year

In [72]:
dfrs['Total_MarkDown'] = dfrs['MarkDown1'] + dfrs['MarkDown2']+ dfrs['MarkDown3'] + dfrs['MarkDown4'] + dfrs['MarkDown5']

In [74]:
store_type_dummies = pd.get_dummies(ds['Type'], prefix = 'Store_Type', drop_first=True)
ds= pd.concat([ds, store_type_dummies], axis = 1)

In [76]:
dt = dt.merge(ds, on='Store', how = 'left')
dt = dt.merge(dfrs, on= ['Store', 'Date'], how='left')
df = df.merge(ds, on='Store',how = 'left')
df= df.merge(dfrs,on =['Store','Date'], how ='left')

In [78]:
dt.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Store_Dept,Month_x,Year_x,Type,Size,...,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Month_y,Year_y,Total_MarkDown
0,1,1,2010-02-05,24924.50,False,1_1,2,2010,A,151315,...,NaN,NaN,NaN,NaN,211.096358,8.106,False,2,2010,NaN
1,1,1,2010-02-12,46039.49,True,1_1,2,2010,A,151315,...,NaN,NaN,NaN,NaN,211.242170,8.106,True,2,2010,NaN
2,1,1,2010-02-19,41595.55,False,1_1,2,2010,A,151315,...,NaN,NaN,NaN,NaN,211.289143,8.106,False,2,2010,NaN
3,1,1,2010-02-26,19403.54,False,1_1,2,2010,A,151315,...,NaN,NaN,NaN,NaN,211.319643,8.106,False,2,2010,NaN
4,1,1,2010-03-05,21827.90,False,1_1,3,2010,A,151315,...,NaN,NaN,NaN,NaN,211.350143,8.106,False,3,2010,NaN


In [80]:
df.head()

,Store,Dept,Date,IsHoliday_x,Store_Dept,Month_x,Year_x,Type,Size,Store_Type_B,...,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Month_y,Year_y,Total_MarkDown
0,1,1,2012-11-02,False,1_1,11,2012,A,151315,False,...,5147.70,50.82,3639.90,2737.42,223.462779,6.573,False,11,2012,18342.28
1,1,1,2012-11-09,False,1_1,11,2012,A,151315,False,...,3370.89,40.28,4646.79,6154.16,223.481307,6.573,False,11,2012,25633.44
2,1,1,2012-11-16,False,1_1,11,2012,A,151315,False,...,292.10,103.78,1133.15,6612.69,223.512911,6.573,False,11,2012,17838.00
3,1,1,2012-11-23,True,1_1,11,2012,A,151315,False,...,4.17,74910.32,209.91,303.32,223.561947,6.573,True,11,2012,76311.31
4,1,1,2012-11-30,False,1_1,11,2012,A,151315,False,...,NaN,3838.35,150.57,6966.34,223.610984,6.573,False,11,2012,NaN


In [82]:
dt['Weekly_Sales_Lag1'] = dt['Weekly_Sales'].shift(1)
dt['Weekly_Sales_Lag2'] = dt['Weekly_Sales'].shift(2)

In [84]:
dt['Rolling_Mean'] = dt['Weekly_Sales'].rolling(window=4).mean()
dt['Rolling_Std'] = dt['Weekly_Sales'].rolling(window=4).std()

In [86]:
dt['Date'] = pd.to_datetime(dt['Date'])

dt['Month']  = dt['Date'].dt.month
dt['Quarter']  = dt['Date'].dt.quarter
dt['WeekOfYear']  = dt['Date'].dt.isocalendar().week

In [88]:
display(dt.head())

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Store_Dept,Month_x,Year_x,Type,Size,...,Month_y,Year_y,Total_MarkDown,Weekly_Sales_Lag1,Weekly_Sales_Lag2,Rolling_Mean,Rolling_Std,Month,Quarter,WeekOfYear
0,1,1,2010-02-05,24924.50,False,1_1,2,2010,A,151315,...,2,2010,NaN,NaN,NaN,NaN,NaN,2,1,5
1,1,1,2010-02-12,46039.49,True,1_1,2,2010,A,151315,...,2,2010,NaN,24924.50,NaN,NaN,NaN,2,1,6
2,1,1,2010-02-19,41595.55,False,1_1,2,2010,A,151315,...,2,2010,NaN,46039.49,24924.50,NaN,NaN,2,1,7
3,1,1,2010-02-26,19403.54,False,1_1,2,2010,A,151315,...,2,2010,NaN,41595.55,46039.49,32990.77,12832.106391,2,1,8
4,1,1,2010-03-05,21827.90,False,1_1,3,2010,A,151315,...,3,2010,NaN,19403.54,41595.55,32216.62,13554.047185,3,1,9


In [90]:
dt['Date'] = pd.to_datetime(dt['Date'])
df['Date'] = pd.to_datetime(df['Date'])
dfrs['Date'] = pd.to_datetime(dfrs['Date'])

In [92]:
dt = dt.sort_values('Date')
df = df.sort_values('Date')
dfrs = dfrs.sort_values('Date')

In [94]:
dt.set_index('Date', inplace=True)
df.set_index('Date', inplace=True)
dfrs.set_index('Date', inplace=True)
dt.fillna(0,inplace=True)
df.fillna(0,inplace=True)
dfrs.fillna(0, inplace=True)
dt.reset_index(inplace=True)
df.reset_index(inplace=True)
dfrs.reset_index(inplace=True)

In [96]:
display(dfrs.head())

,Date,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Month,Year,Total_MarkDown
0,2010-02-05,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,2,2010,0.0
1,2010-02-05,16,19.79,2.580,0.0,0.0,0.0,0.0,0.0,189.381697,7.039,False,2,2010,0.0
2,2010-02-05,31,39.05,2.572,0.0,0.0,0.0,0.0,0.0,210.752605,8.324,False,2,2010,0.0
3,2010-02-05,21,39.05,2.572,0.0,0.0,0.0,0.0,0.0,210.752605,8.324,False,2,2010,0.0
4,2010-02-05,26,9.55,2.788,0.0,0.0,0.0,0.0,0.0,131.527903,8.488,False,2,2010,0.0


In [98]:
dt = dt.infer_objects(copy=False)
df = df.infer_objects(copy=False)
dt.loc[:, dt.select_dtypes(include=[np.number]).columns] = dt.select_dtypes(include=[np.number]).interpolate(method='linear')
df.loc[:, df.select_dtypes(include=[np.number]).columns] = df.select_dtypes(include=[np.number]).interpolate(method='linear')

In [103]:
print(dt.head())

        Date  Store  Dept  Weekly_Sales  IsHoliday_x Store_Dept  Month_x  \
0 2010-02-05      1     1      24924.50        False        1_1        2   
1 2010-02-05     29     5      15552.08        False       29_5        2   
2 2010-02-05     29     6       3200.22        False       29_6        2   
3 2010-02-05     29     7      10820.05        False       29_7        2   
4 2010-02-05     29     8      20055.64        False       29_8        2   

   Year_x Type    Size  ...  Weekly_Sales_Lag2  Rolling_Mean  Rolling_Std  \
0    2010    A  151315  ...               0.00        0.0000     0.000000   
1    2010    B   93638  ...           15520.36    15721.8300   538.964060   
2    2010    B   93638  ...           17657.20    10061.1675  5939.907431   
3    2010    B   93638  ...            1356.70     3806.6575  4691.480600   
4    2010    B   93638  ...           13066.83    14640.0725  3805.271794   

   Month  Quarter  WeekOfYear  Is_Holiday  Avg_Temperature  sin_month  \
0      